<a href="https://colab.research.google.com/github/svondracek0/Data-Science-Practicum/blob/main/Vondracek_hw09_dspract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP DS Praktikum2021 HW02 - Generátor textu

*Štěpán Vondráček*

Na souboru https://github.com/ryanmcdermott/trump-speeches/blob/master/speeches.txt natrénuji transformer, který následně vygeneruje náhodný Trumpův projev. Můj projekt je do značné míry založen na projektu zabývajícím se projevy A. Babiše.




# 1 Inicializace Working Environmentu

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import re

from google.colab import drive

In [ ]:

# Nacteni souboru trump_speech z google disku

def open_txt(path = "/content/drive/MyDrive/trump_speech.txt"):
  file = open(path, "r")
  data = file.read()
  return data
  file.close()

data = open_txt() #texty mame nactene po radcich do seznamu 




## 2 Preprocessing

In [ ]:
#pomoci regexp nahradim prazdne radky a nddbytecne whitespaces za vetami teckou
data = data.replace('\n', "").replace('. ', '.')
#nyni rozdelim text tak, ze jednotlive vety budou tvorit jeden prvek pole
while m:
  position = m.span()[0]
  data = data[:position] + "&" + data[position + 1 :]
  m = re.search('\.[A-Z]', data)
corpus = data.lower().split('&')
print(corpus[:5])



In [ ]:
#tokenizace
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print("Total words: ", total_words)

input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i + 1]
    input_sequences.append(n_gram_sequence)
print(input_sequences[1:10])







In [ ]:
# padding
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
print(input_sequences[1:5])
print(max_sequence_len)

In [ ]:
xs = input_sequences[:, :-1]
labels = input_sequences[:, -1]
ys = tf.keras.utils.to_categorical(labels, num_classes = total_words)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
#model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=10, verbose=1)
#print model.summary()
print(model)

In [ ]:
seed_text = We have drained the swamp in Washington D.C.!!!
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)